In [4]:
import pandas as pd 
df = pd.read_csv("plataformas.csv", sep=",")

Pelicula con mayor duracion según año, plataforma y tipo de duración

In [4]:
def get_max_duration(year: int, platform: str, duration_type: str):  
    # Cargar el archivo CSV en un DataFrame. Y le aclaro en parse_dates para indicarle a pandas que esa columna debe ser tratada como fecha en ves de cadena de caracteres
    df = pd.read_csv("plataformas.csv", parse_dates=["date_added"]) 
     
    # Voy a indicarle en plataforma que me interesa la primer letra que es como le indique a la hora de crearlo para cada plataforma. (Ej:Netflix = n)
    # Tambien si el usuario ingresa en mayuscula que lo cambie a minuscula 
    duration_type = duration_type.lower() 
    platfrom = platform.lower()[0] 
     
    #Filtro por año de lanzamiento y por plataforma. Que tenga encuentra la primer letra de id 
    df = df[(df["release_year"] == year) & (df["id"].str.startswith(platfrom)) & (df["duration_type"] == duration_type) & (df["type"] == "movie")] 

    #En la condicion voy a indicar tanto la palabra en ingles como español
    if duration_type == "min" or duration_type == "duration" or duration_type == "duracion": 
       #Filtro por duracion y que devuelva todos los datos
       df_duration = df[df["duration_type"] == duration_type] 
       max_duration = df_duration["duration_int"].max()   
       max_duration_data = df_duration.loc[df_duration["duration_int"] == max_duration] 
     
    # En este caso lo mismo, indico tanto el ingreso en ingles como español
    elif duration_type == "seasons" or duration_type == "temporada": 
        df_duration = df[df["duration_type"] == duration_type] 
        max_duration = df_duration["duration_int"].max()   
        max_duration_data = df_duration.loc[df_duration["duration_int"] == max_duration]  
      
    # Selecciono el primer registro de max_duration_date y obtengo el titulo que cumplen la condicion
    title = max_duration_data.iloc[0]["title"]
    
    #Retorno el titulo de la pelicula con mayor duracion. Ya que en max_duration_data tenia toda la informacion de esa pelicula y solo me pedia devolver el titulo
    return title

In [5]:
#Ejecuto la funcion 
get_max_duration(2019,"NETFLIX","min")

'this earth of mankind'

Cantidad de películas según plataforma, con un puntaje mayor a XX en determinado año.

In [6]:
def get_score_count(platform: str, scored: float, year: int) -> int: 
      
    df = pd.read_csv("plataformas.csv", parse_dates=["date_added"]) 
     
    # Convierto a minúsculas y obtengo la primera letra de la plataforma  
    platform = platform.lower()[0] 
     
    #Filtro el df para que sea pelicula solamente,  
    # selecciono todas las filas que tienen el valor de id que comienza con la letra que representa la plataforma(startswith)  
    # Que el valor sea mayor al de scored 
    # Y el año de la pelicula que se paso en la funcion
    df_filtrado = df.loc[(df["type"] == "movie") & (df["id"].str.startswith(platform)) & (df["scored"] > scored) & (df["release_year"] == year)]

    # Devolver la cantidad de filas del DataFrame resultante 
    return len(df_filtrado)

In [7]:
get_score_count("netflix", 1.5, 2019)

226

Cantidad de películas debe devolver un int, con el número total de películas de esa plataforma. Las plataformas deben llamarse amazon, netflix, hulu, disney.

In [8]:
def get_count_platform(platform):  
    df = pd.read_csv("plataformas.csv", parse_dates=["date_added"]) 
    #Como anteriormente me voy a asegurar que lo que se ingrese lo pase a minusculas 
    platform = platform.lower() 
    platform = platform.lower()[0]   
    #filtro 
    df_filtrado = df.loc[(df["type"] == "movie") & (df["id"].str.startswith(platform))] 
    #Ahora cuento la cantidad de filas del df_filtrado con las condiciones y selecciono el primer elemento 
    count = df_filtrado.shape[0] 
    return count

In [9]:
get_count_platform("HULU")

1484

Actor que más se repite según plataforma y año. Debo devolver el nombre del actor que más se repite según la plataforma y el año dado.

In [10]:
def get_actor(platform: str, year: int):
     
    df = pd.read_csv("plataformas.csv", parse_dates=["date_added"])

    # Filtro por plataforma y año
    platform = platform.lower()[0]
    df = df[(df["id"].str.startswith(platform)) & (df["release_year"] == year) & (df["type"] == "movie")]

    # filtro una lista con todos los actores 
    #Utilizo flatten para obtener un array de numpy unidimensional con todos los valores.  
    #De esta manera, obtengo una lista de todos los actores presentes en el dataframe, sin importar en qué columna aparezcan.
    actores = df["cast"].str.split(", ", expand=True).values.flatten() 
     
    # Compruebo si la lista de actores está vacía O tenemos valores en NaN. Si esta condicion se cumple devuelvo "no hay datos".
    if actores.size == 0 or pd.isnull(actores).all():
        return "no hay datos"

    # Contar la cantidad de veces que aparece cada actor
    cantidad_actores = pd.Series(actores).value_counts()

    # Obtener el actor más repetido
    top_actor = cantidad_actores.idxmax()

    # Devolver el nombre del actor
    return top_actor

In [11]:
get_actor("amazon_prime", 2000)

'kathleen turner'

La cantidad de contenidos/productos que se publicó por país y año. Debo devolver el tipo de contenido (pelicula,serie,documental) por pais y año en un diccionario con las variables llamadas 'pais' (nombre del pais), 'anio' (año), 'pelicula' (tipo de contenido).

In [12]:
def prod_per_country(tipo: str, pais: str, anio: int):

    # Filtro por tipo de contenido, país y año.
    data = df[(df["type"] == tipo.lower()) & (df["country"] == pais) & (df["release_year"] == anio)]

    # Agrupo los datos por país, año y tipo de contenido, y cuento la cantidad de productos en cada grupo. 
    #Un count del grupo para que me cuente la cantidad de veces que aparece cada combinacion unica de las 3 columnas
    #Reset_index elimino el indice actual y lo reemplazo por una secuencia numerica para hacer mas facil contarlos
    cantidad_producto = data.groupby(["country", "release_year", "type"]).count()["id"].reset_index()

    # Creo un diccionario con los resultados y lo devuelvo.
    resultado = {
        "pais": pais,
        "anio": anio,
        "contenido": tipo, 
        # Accedo al valor de la columna id de la primera fila del df can tidad_producto. Y despues si la cantidad es mayor a 0 muestro la cantidad si no 0.
        "cantidad": int(cantidad_producto.iloc[0]["id"]) if len(cantidad_producto) > 0 else 0
    }

    return resultado

In [13]:
prod_per_country("seasons","south korea", 2015)

{'pais': 'south korea', 'anio': 2015, 'contenido': 'seasons', 'cantidad': 0}

Esto quiere decir que en south korea en el año 2017 hubo 3 tv show.

La cantidad total de contenidos/productos (todo lo disponible en streaming, series, documentales, peliculas, etc) según el rating de audiencia dado (para que publico fue clasificada la pelicula). La función debe llamarse get_contents(rating) y debe devolver el numero total de contenido con ese rating de audiencias.

In [14]:
def get_contents(rating: str):
    # Cargar el archivo CSV en un DataFrame.
    df = pd.read_csv("plataformas.csv", parse_dates=["date_added"]) 

    # Filtro por rating de audiencia.
    df = df[df["rating"] == rating]

    # Cuento la cantidad de productos que cumplen con la condición.
    numero_contents = len(df)

    return numero_contents

In [15]:
get_contents("g")

1231